In [1]:
from warnings import filterwarnings
filterwarnings("ignore")

# Read data

In [2]:
import pandas as pd
A=pd.read_csv("C:/Users/Hp/Downloads/train.csv")

In [3]:
A.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


# Missing Data Treatment & Exploratory Data Analysis

In [4]:
from PM6 import chisq,ANOVA,replacer
replacer(A)
q = list(A.columns)
q.remove("Item_Outlet_Sales")

imp_con = []
imp_cat = []

for i in q:
    if(A[i].dtypes=="object"):
        print("--------------------")
        print("Item_Outlet_Sales vs",i)
        w = chisq(A,"Item_Outlet_Sales",i)
        print(w)
        if(w < 0.05):
            imp_cat.append(i)
    else:       
        print("--------------------")
        print("Item_Outlet_Sales vs",i)
        w = ANOVA(A,"Item_Outlet_Sales",i)
        print(w)
        if(w < 0.05):
            imp_con.append(i)

--------------------
Item_Outlet_Sales vs Item_Identifier
0.0
--------------------
Item_Outlet_Sales vs Item_Weight
0.2863
--------------------
Item_Outlet_Sales vs Item_Fat_Content
0.6543311172300722
--------------------
Item_Outlet_Sales vs Item_Visibility
0.0
--------------------
Item_Outlet_Sales vs Item_Type
1.0335983028373673e-28
--------------------
Item_Outlet_Sales vs Item_MRP
0.0
--------------------
Item_Outlet_Sales vs Outlet_Identifier
3.7619710870486675e-27
--------------------
Item_Outlet_Sales vs Outlet_Establishment_Year
0.0
--------------------
Item_Outlet_Sales vs Outlet_Size
0.6070208632583967
--------------------
Item_Outlet_Sales vs Outlet_Location_Type
0.035581810296713895
--------------------
Item_Outlet_Sales vs Outlet_Type
3.4371967318312466e-121


In [5]:
imp_cat

['Item_Identifier',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Location_Type',
 'Outlet_Type']

In [6]:
imp_con

['Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year']

In [7]:
imp_cols=imp_con+imp_cat

In [8]:
imp_cols

['Item_Visibility',
 'Item_MRP',
 'Outlet_Establishment_Year',
 'Item_Identifier',
 'Item_Type',
 'Outlet_Identifier',
 'Outlet_Location_Type',
 'Outlet_Type']

# Define X & Y

In [9]:
X=A[imp_cols]

In [10]:
X

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type
0,0.016047,249.8092,1999,FDA15,Dairy,OUT049,Tier 1,Supermarket Type1
1,0.019278,48.2692,2009,DRC01,Soft Drinks,OUT018,Tier 3,Supermarket Type2
2,0.016760,141.6180,1999,FDN15,Meat,OUT049,Tier 1,Supermarket Type1
3,0.000000,182.0950,1998,FDX07,Fruits and Vegetables,OUT010,Tier 3,Grocery Store
4,0.000000,53.8614,1987,NCD19,Household,OUT013,Tier 3,Supermarket Type1
...,...,...,...,...,...,...,...,...
8518,0.056783,214.5218,1987,FDF22,Snack Foods,OUT013,Tier 3,Supermarket Type1
8519,0.046982,108.1570,2002,FDS36,Baking Goods,OUT045,Tier 2,Supermarket Type1
8520,0.035186,85.1224,2004,NCJ29,Health and Hygiene,OUT035,Tier 2,Supermarket Type1
8521,0.145221,103.1332,2009,FDN46,Snack Foods,OUT018,Tier 3,Supermarket Type2


In [11]:
X.columns

Index(['Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year',
       'Item_Identifier', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type'],
      dtype='object')

In [12]:
Y=A[["Item_Outlet_Sales"]]

# Preprocessing 

In [13]:
from PM6 import preprocessing
Xnew=preprocessing(X)

In [14]:
Xnew

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier_DRA12,Item_Identifier_DRA24,Item_Identifier_DRA59,Item_Identifier_DRB01,Item_Identifier_DRB13,Item_Identifier_DRB24,Item_Identifier_DRB25,...,Outlet_Identifier_OUT045,Outlet_Identifier_OUT046,Outlet_Identifier_OUT049,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,-0.970732,1.747454,0.139541,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
1,-0.908111,-1.489023,1.334103,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,-0.956917,0.010040,0.139541,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,1,0,0
3,-1.281758,0.660050,0.020085,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,-1.281758,-1.399220,-1.293934,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,-0.181193,1.180783,-1.293934,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
8519,-0.371154,-0.527301,0.497909,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,0,0
8520,-0.599784,-0.897208,0.736822,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
8521,1.532880,-0.607977,1.334103,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


# Divide the data in training & testing set

In [15]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(Xnew,Y,test_size=0.2,random_state=21)

# Linear Regression model

In [16]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
model=lr.fit(xtrain,ytrain)
pred_ts=model.predict(xtest)
pred_tr=model.predict(xtrain)

In [17]:
from sklearn.metrics import mean_squared_error
before_bias=mean_squared_error(ytrain,pred_tr)
before_variance=mean_squared_error(ytest,pred_ts)

In [18]:
before_bias

980338.1033837342

In [19]:
before_variance

7.051085599093014e+25

# Regularize

In [20]:
values_to_try=[]
x=1
for i in range(0,50):
    x=x-0.001
    values_to_try.append(round(x,3))
for i in range(0,50):
    x=x+0.001
    values_to_try.append(round(x,3))

In [21]:
tunning_grid={"alpha":values_to_try}

# Ridge Regression

In [22]:
from sklearn.linear_model import Ridge
rr=Ridge()
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(rr,tunning_grid,scoring="neg_mean_squared_error",cv=4)

cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 1.0}

In [23]:
rr=Ridge(alpha=1.0)
model=rr.fit(xtrain,ytrain)

pred_ts=model.predict(xtest)
pred_tr=model.predict(xtrain)

from sklearn.metrics import mean_squared_error
bias=mean_squared_error(ytrain,pred_tr)
variance=mean_squared_error(ytest,pred_ts)
print(bias,variance)

994425.0141684263 1493270.9587571595


# Lasso

In [24]:
from sklearn.linear_model import Lasso
ls=Lasso()
from sklearn.model_selection import GridSearchCV
cv=GridSearchCV(ls,tunning_grid,scoring="neg_mean_squared_error",cv=4)

cvmodel=cv.fit(xtrain,ytrain)
cvmodel.best_params_

{'alpha': 1.0}

In [25]:
ls=Lasso(alpha=1.0)
model=ls.fit(xtrain,ytrain)

pred_ts=model.predict(xtest)
pred_tr=model.predict(xtrain)

from sklearn.metrics import mean_squared_error
bias=mean_squared_error(ytrain,pred_tr)
variance=mean_squared_error(ytest,pred_ts)
print(bias,variance)

1264432.5627439592 1265951.7896546545


In [26]:
#we will consider Lasso as the testing error i.e variance is lesser

# Create Prediction on testing data

In [27]:
B=pd.read_csv("C:/Users/Hp/Downloads/Test.csv")

In [28]:
B.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [29]:
replacer(B)

In [30]:
tsd=preprocessing(B)

In [31]:
Q = list(Xnew.columns)
for i in ['Item_Identifier_DRE49', 'Item_Identifier_FDA04', 'Item_Identifier_FDG24', 'Item_Identifier_FDG33', 'Item_Identifier_FDL10', 'Item_Identifier_FDL34', 'Item_Identifier_FDO19', 'Item_Identifier_FDO52', 'Item_Identifier_FDT07', 'Item_Identifier_FDU19', 'Item_Identifier_FDW13', 'Item_Identifier_FDX04', 'Item_Identifier_FDX20', 'Item_Identifier_NCL31', 'Item_Identifier_NCQ06', 'Item_Identifier_NCY18']:
    tsd[i]=0

In [32]:
final=tsd[Q]

In [33]:
final.shape

(5681, 1595)

In [34]:
Xnew.shape

(8523, 1595)

In [35]:
from sklearn.linear_model import Lasso

ls=Lasso(alpha=1.0)
model=ls.fit(Xnew,Y)

predicted=model.predict(final)

In [36]:
predicted

array([1844.52059181, 1498.2901808 , 1930.45100698, ..., 1846.72480591,
       3547.8496164 , 1253.83833488])

In [37]:
T=B[['Item_Visibility', 'Item_MRP', 'Outlet_Establishment_Year',
       'Item_Identifier', 'Item_Type', 'Outlet_Identifier',
       'Outlet_Location_Type', 'Outlet_Type']]
T["Item_Outlet_Sales"]=predicted

In [38]:
T

,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Identifier,Item_Type,Outlet_Identifier,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,0.007565,107.8622,1999,FDW58,Snack Foods,OUT049,Tier 1,Supermarket Type1,1844.520592
1,0.038428,87.3198,2007,FDW14,Dairy,OUT017,Tier 2,Supermarket Type1,1498.290181
2,0.099575,241.7538,1998,NCN55,Others,OUT010,Tier 3,Grocery Store,1930.451007
3,0.015388,155.0340,2007,FDQ58,Snack Foods,OUT017,Tier 2,Supermarket Type1,2587.816679
4,0.118599,234.2300,1985,FDY38,Dairy,OUT027,Tier 3,Supermarket Type3,5121.439842
...,...,...,...,...,...,...,...,...,...
5676,0.013496,141.3154,1997,FDB58,Snack Foods,OUT046,Tier 1,Supermarket Type1,2287.789563
5677,0.142991,169.1448,2009,FDD47,Starchy Foods,OUT018,Tier 3,Supermarket Type2,2410.931448
5678,0.073529,118.7440,2002,NCO17,Health and Hygiene,OUT045,Tier 2,Supermarket Type1,1846.724806
5679,0.000000,214.6218,2007,FDJ26,Canned,OUT017,Tier 2,Supermarket Type1,3547.849616


In [39]:
T.to_csv("C:/Users/Hp/Downloads/Item_Outlet_Sales.csv")

# End